In [25]:
import numpy as np
from scipy import linalg
from time import time

def logistic(x, y, num_iteration=1000, learning_rate=1e-3):
    r, c = x.shape
    p = c + 1
    X = np.hstack((np.ones((r,1)), x))
    beta = 2*np.random.randn(p, 1)-1
    for i in range(num_iteration):
        pr = sigmoid(X.dot(beta))
        beta = beta + learning_rate* X.T.dot(y-pr)
    return beta

def logistic_polyak(X, y, num_iterations=1000, learning_rate=1e-2, momentum=0.9):
    r, c = X.shape
    X = np.hstack((np.ones((r, 1)), X))
    beta = 2 * np.random.randn(c + 1, 1) - 1
    velocity = np.zeros_like(beta)

    for i in range(num_iterations):
        predictions = sigmoid(X.dot(beta))
        gradient = X.T.dot(y - predictions)
        velocity = momentum * velocity + learning_rate * gradient
        beta += velocity

    return beta

def logistic_nesterov(X, y, num_iterations=1000, learning_rate=1e-2, momentum=0.9):
    r, c = X.shape
    X = np.hstack((np.ones((r, 1)), X))
    beta = 2 * np.random.randn(c + 1, 1) - 1
    velocity = np.zeros_like(beta)

    for i in range(num_iterations):
        # Lookahead position
        lookahead_beta = beta + momentum * velocity
        predictions = sigmoid(X.dot(lookahead_beta))
        gradient = X.T.dot(y - predictions)
        velocity = momentum * velocity + learning_rate * gradient
        beta += velocity

    return beta

def sigmoid(x):
    return 1.0 / (1 + np.exp(-x))


In [10]:
from sklearn.model_selection import train_test_split
np.random.seed(42)
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
X = data.data
y = data.target.reshape(-1, 1) 

In [11]:
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,y,train_size=0.8,random_state=True)

In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(Xtrain)
X_test_scaled = scaler.transform(Xtest)

In [36]:
def predict(x, beta, threshold=0.5):
    X = np.hstack((np.ones((x.shape[0], 1)), x))
    probabilities = sigmoid(X.dot(beta))
    return (probabilities >= threshold).astype(int)
start =time()
logistic_beta = logistic(X_train_scaled, Ytrain)
stop1 = time() - start
print(stop1)
start =time()
nestorov_beta = logistic_nesterov(X_train_scaled, Ytrain)
stop2 = time() - start
print(stop2)
start = time()
polyak_beta = logistic_polyak(X_train_scaled, Ytrain)
stop3 = time() - start
print(stop3)
Ypred = predict(X_test_scaled, logistic_beta)
Ypred_nestorov = predict(X_test_scaled,nestorov_beta)
Ypred_polyak = predict(X_test_scaled,polyak_beta)
accuracy = np.mean(Ypred == Ytest)
accuracy_nest = np.mean(Ypred_nestorov==Ytest)
accuracy_pol = np.mean(Ypred_polyak==Ytest)
print(f"Accuracy on test data: {accuracy:.8f}")
print(f"Accuracy on test data: {accuracy_nest:.8f}")
print(f"Accuracy on test data: {accuracy_pol:.8f}")

0.03271222114562988
0.022781848907470703
0.017239093780517578
Accuracy on test data: 0.98245614
Accuracy on test data: 0.96491228
Accuracy on test data: 0.96491228
